In [43]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir(r"dataset\frames\Train")

label_types = os.listdir(r"dataset\frames\Train")
print (label_types)  

['a', 'a lot', 'abdomen', 'able', 'about', 'above', 'accent', 'accept', 'accident', 'accomplish', 'accountant', 'across', 'act', 'action', 'active', 'activity', 'actor', 'adapt', 'add', 'address', 'adjective', 'adjust', 'admire', 'admit', 'adopt', 'adult', 'advanced', 'advantage', 'adverb', 'affect', 'afraid', 'africa', 'after', 'afternoon', 'again', 'against', 'age', 'agenda', 'ago', 'agree', 'agreement', 'ahead', 'aid', 'aim', 'airplane', 'alarm', 'alcohol', 'algebra', 'all', 'all day', 'allergy', 'alligator', 'allow', 'almost', 'alone', 'alphabet', 'already', 'also', 'always', 'amazing', 'america', 'amputate', 'analyze', 'anatomy', 'and', 'angel', 'angle', 'angry', 'animal', 'anniversary', 'announce', 'annoy', 'another', 'answer', 'any', 'anyway', 'apart', 'apartment', 'apostrophe', 'appear', 'appetite', 'apple', 'appointment', 'appreciate', 'approach', 'appropriate', 'approve', 'april', 'archery', 'area', 'argue', 'arizona', 'arm', 'army', 'around', 'arrest', 'arrive', 'arrogant', 

In [44]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r"dataset\frames\Train" + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r"dataset\frames\Train" + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

     tag                        video_name
0      a      dataset\frames\Train/a/01610
1      a      dataset\frames\Train/a/01612
2      a      dataset\frames\Train/a/01615
3  a lot  dataset\frames\Train/a lot/02124
4  a lot  dataset\frames\Train/a lot/02125
           tag                           video_name
8308  yourself  dataset\frames\Train/yourself/64447
8309  yourself  dataset\frames\Train/yourself/64451
8310      zero      dataset\frames\Train/zero/64544
8311      zero      dataset\frames\Train/zero/64547
8312      zero      dataset\frames\Train/zero/66825


In [45]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [46]:
dataset_path = os.listdir(r'dataset\frames\Test')
print(dataset_path)

room_types = os.listdir(r'dataset\frames\Test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir(r'dataset\frames\Test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(r'dataset\frames\Test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['a', 'a lot', 'abdomen', 'able', 'about', 'above', 'accent', 'accept', 'accident', 'accomplish', 'accountant', 'across', 'act', 'action', 'active', 'activity', 'actor', 'adapt', 'add', 'address', 'adjective', 'adjust', 'admire', 'admit', 'adopt', 'adult', 'advanced', 'advantage', 'adverb', 'affect', 'afraid', 'africa', 'after', 'afternoon', 'again', 'against', 'age', 'agenda', 'ago', 'agree', 'agreement', 'ahead', 'aid', 'aim', 'airplane', 'alarm', 'alcohol', 'algebra', 'all', 'allergy', 'alligator', 'allow', 'almost', 'alone', 'alphabet', 'already', 'also', 'always', 'amazing', 'america', 'amputate', 'analyze', 'anatomy', 'and', 'angel', 'angle', 'angry', 'animal', 'announce', 'another', 'answer', 'any', 'anyway', 'apart', 'apartment', 'apostrophe', 'appear', 'appetite', 'apple', 'appointment', 'appreciate', 'approach', 'appropriate', 'approve', 'april', 'archery', 'area', 'argue', 'arizona', 'arm', 'army', 'around', 'arrest', 'arrive', 'arrogant', 'art', 'article', 'artist', 'asia',

In [47]:
pip install tensorflow_docs --user

Note: you may need to restart the kernel to use updated packages.


In [48]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [49]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [50]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 8313
Total videos for testing: 3667


,Unnamed: 0,video_name,tag
7204,7204,dataset\frames\Train/talent/56793,talent
2442,2442,dataset\frames\Train/drive/17762,drive
1617,1617,dataset\frames\Train/comfortable/65381,comfortable
5252,5252,dataset\frames\Train/pepper/41943,pepper
1377,1377,dataset\frames\Train/chat/10112,chat
54,54,dataset\frames\Train/act/00849,act
4540,4540,dataset\frames\Train/medicine/35455,medicine
2289,2289,dataset\frames\Train/discuss/16592,discuss
1381,1381,dataset\frames\Train/cheap/10141,cheap
1279,1279,dataset\frames\Train/category/09489,category


In [51]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [52]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [58]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels


['a', 'a lot', 'abdomen', 'able', 'about', 'above', 'accent', 'accept', 'accident', 'accomplish', 'accountant', 'across', 'act', 'action', 'active', 'activity', 'actor', 'adapt', 'add', 'address', 'adjective', 'adjust', 'admire', 'admit', 'adopt', 'adult', 'advanced', 'advantage', 'adverb', 'affect', 'afraid', 'africa', 'after', 'afternoon', 'again', 'against', 'age', 'agenda', 'ago', 'agree', 'agreement', 'ahead', 'aid', 'aim', 'airplane', 'alarm', 'alcohol', 'algebra', 'all', 'all day', 'allergy', 'alligator', 'allow', 'almost', 'alone', 'alphabet', 'already', 'also', 'always', 'amazing', 'america', 'amputate', 'analyze', 'anatomy', 'and', 'angel', 'angle', 'angry', 'animal', 'anniversary', 'announce', 'annoy', 'another', 'answer', 'any', 'anyway', 'apart', 'apartment', 'apostrophe', 'appear', 'appetite', 'apple', 'appointment', 'appreciate', 'approach', 'appropriate', 'approve', 'april', 'archery', 'area', 'argue', 'arizona', 'arm', 'army', 'around', 'arrest', 'arrive', 'arrogant', 

array([[   0],
       [   0],
       [   0],
       ...,
       [1997],
       [1997],
       [1997]], dtype=int64)

In [59]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [63]:
import tensorflow as tf

# Define your StringLookup layer
label_processor = tf.keras.layers.StringLookup(
    vocabulary=['a', 'a lot', 'abdomen', 'able', 'about', 'above', 'accent', 'accept', 'accident', 'accomplish', 'accountant', 'across', 'act', 'action', 'active', 'activity', 'actor', 'adapt', 'add', 'address', 'adjective', 'adjust', 'admire', 'admit', 'adopt', 'adult', 'advanced', 'advantage', 'adverb', 'affect', 'afraid', 'africa', 'after', 'afternoon', 'again', 'against', 'age', 'agenda', 'ago', 'agree', 'agreement', 'ahead', 'aid', 'aim', 'airplane', 'alarm', 'alcohol', 'algebra', 'all', 'all day', 'allergy', 'alligator', 'allow', 'almost', 'alone', 'alphabet', 'already', 'also', 'always', 'amazing', 'america', 'amputate', 'analyze', 'anatomy', 'and', 'angel', 'angle', 'angry', 'animal', 'anniversary', 'announce', 'annoy', 'another', 'answer', 'any', 'anyway', 'apart', 'apartment', 'apostrophe', 'appear', 'appetite', 'apple', 'appointment', 'appreciate', 'approach', 'appropriate', 'approve', 'april', 'archery', 'area', 'argue', 'arizona', 'arm', 'army', 'around', 'arrest', 'arrive', 'arrogant', 'art', 'article', 'artist', 'asia', 'ask', 'asl', 'assist', 'assistant', 'assume', 'attend', 'attention', 'attitude', 'attorney', 'attract', 'auction', 'audience', 'audiologist', 'audiology', 'august', 'aunt', 'australia', 'austria', 'author', 'authority', 'autumn', 'available', 'average', 'avoid', 'awake', 'award', 'aware', 'away', 'awful', 'awkward', 'b', 'baby', 'babysitter', 'back', 'background', 'backpack', 'bacon', 'bad', 'bake', 'balance', 'bald', 'ball', 'balloon', 'banana', 'bank', 'baptize', 'bar', 'barely', 'bark', 'baseball', 'basement', 'basic', 'basketball', 'bath', 'bathroom', 'battery', 'battle', 'bear', 'beard', 'beautiful', 'because', 'become', 'bed', 'bedroom', 'bee', 'beer', 'before', 'beg', 'beginning', 'behavior', 'behind', 'belief', 'believe', 'bell', 'below', 'belt', 'benefit', 'berry', 'beside', 'best', 'bet', 'better', 'between', 'bible', 'bicycle', 'big', 'bike', 'binoculars', 'biology', 'bird', 'birth', 'birthday', 'bite', 'bitter', 'black', 'blame', 'blanket', 'blend', 'bless', 'blind', 'blood', 'blow', 'blue', 'boast', 'boat', 'body', 'boil', 'bone', 'book', 'bookshelf', 'bookstore', 'boots', 'bored', 'borrow', 'boss', 'both', 'bother', 'bottle', 'bottom', 'bowl', 'bowling', 'box', 'boxing', 'boy', 'boyfriend', 'bra', 'bracelet', 'brag', 'braid', 'brain', 'brave', 'bread', 'break', 'breakdown', 'breakfast', 'breathe', 'breeze', 'bribe', 'bridge', 'brief', 'bright', 'bring', 'brochure', 'broke', 'brother', 'brown', 'brush', 'buffalo', 'bug', 'build', 'building', 'bull', 'bully', 'burp', 'bus', 'bush', 'business', 'busy', 'but', 'butter', 'butterfly', 'button', 'buy', 'bye', 'cabbage', 'cabinet', 'cafeteria', 'cake', 'calculate', 'calculator', 'calculus', 'california', 'call', 'calm', 'camel', 'camera', 'camp', 'camping', 'can', 'canada', 'cancel', 'candidate', 'candle', 'candy', 'cannot', 'canoe', 'captain', 'caption', 'car', 'card', 'cards', 'care', 'careful', 'careless', 'carnival', 'carrot', 'carry', 'cat', 'catch', 'category', 'caterpillar', 'catholic', 'cause', 'ceiling', 'celebrate', 'cemetery', 'cent', 'center', 'cereal', 'certificate', 'chain', 'chair', 'challenge', 'champion', 'chance', 'change', 'chapter', 'character', 'chase', 'chat', 'cheap', 'cheat', 'check', 'cheerleader', 'cheese', 'chemical', 'chemistry', 'cherry', 'chicago', 'chicken', 'child', 'children', 'china', 'chocolate', 'choice', 'choir', 'choke', 'choose', 'chop', 'christ', 'christian', 'christmas', 'church', 'cigarette', 'circle', 'city', 'class', 'classroom', 'clean', 'clear', 'clever', 'click', 'climb', 'clock', 'close', 'closet', 'clothes', 'cloud', 'clown', 'clueless', 'clumsy', 'coach', 'coat', 'cochlear implant', 'coconut', 'coffee', 'cold', 'collect', 'college', 'color', 'comb', 'come', 'come here', 'comfortable', 'comma', 'command', 'comment', 'commit', 'committee', 'common', 'common sense', 'community', 'commute', 'company', 'compare', 'compete', 'complain', 'complete', 'complex', 'compromise', 'computer', 'concentrate', 'concept', 'concern', 'conflict', 'confront', 'confused', 'congratulations', 'congress', 'connect', 'conquer', 'consider', 'constitution', 'construct', 'consume', 'contact', 'contest', 'continue', 'contract', 'contribute', 'control', 'conversation', 'convert', 'convince', 'cook', 'cookie', 'cool', 'cooperate', 'cop', 'copy', 'corn', 'corner', 'correct', 'cost', 'couch', 'cough', 'counsel', 'counselor', 'count', 'country', 'court', 'cousin', 'cover', 'cow', 'crab', 'cracker', 'crash', 'crave', 'crazy', 'create', 'crocodile', 'cross', 'crown', 'cruel', 'crush', 'cry', 'cuba', 'culture', 'cup', 'curious', 'curriculum', 'curse', 'curtain', 'customer', 'cut', 'cute', 'd', 'dad', 'daily', 'damage', 'dance', 'dancer', 'danger', 'dangerous', 'dark', 'date', 'daughter', 'dawn', 'day', 'dead', 'deaf', 'death', 'debate', 'debt', 'december', 'decide', 'decorate', 'decrease', 'deduct', 'deep', 'deer', 'defeat', 'defend', 'degree', 'delay', 'delicious', 'deliver', 'demand', 'democrat', 'demonstrate', 'dentist', 'deny', 'deodorant', 'department', 'depend', 'deposit', 'depressed', 'descend', 'describe', 'desert', 'design', 'desk', 'dessert', 'destroy', 'detach', 'detective', 'determine', 'develop', 'devil', 'diabetes', 'diamond', 'diaper', 'diarrhea', 'dice', 'dictionary', 'die', 'different', 'difficult', 'dig', 'dime', 'dining room', 'dinner', 'dinosaur', 'diploma', 'director', 'dirt', 'dirty', 'disagree', 'disappear', 'discipline', 'disconnect', 'discount', 'discover', 'discuss', 'disgust', 'disgusted', 'dismiss', 'dissolve', 'disturb', 'dive', 'divide', 'division', 'divorce', 'dizzy', 'doctor', 'document', 'dog', 'doll', 'dollar', 'dolphin', "don't want", 'done', 'door', 'dorm', 'dormitory', 'double', 'doubt', 'down', 'downstairs', 'drag', 'dragon', 'drama', 'draw', 'drawer', 'dream', 'dress', 'drink', 'drive', 'drop', 'drug', 'drum', 'drunk', 'dry', 'duck', 'due', 'dull', 'dumb', 'during', 'dusk', 'duty', 'dvd', 'dye', 'e', 'each', 'eagle', 'ear', 'early', 'earn', 'earring', 'earth', 'earthquake', 'east', 'easter', 'easy', 'eat', 'economy', 'educate', 'education', 'effort', 'egg', 'egypt', 'eight', 'eighteen', 'either', 'electrician', 'electricity', 'elementary', 'elephant', 'elevator', 'else', 'email', 'embarrass', 'emergency', 'emotion', 'empty', 'encourage', 'end', 'energy', 'engage', 'engagement', 'engine', 'engineer', 'england', 'english', 'enjoy', 'enormous', 'enough', 'enter', 'envelope', 'environment', 'equal', 'erase', 'eraser', 'escape', 'establish', 'eternity', 'europe', 'evaluate', 'evening', 'event', 'every', 'every monday', 'every tuesday', 'everyday', 'everything', 'evidence', 'exact', 'exaggerate', 'example', 'except', 'exchange', 'excited', 'excuse', 'exercise', 'exhibit', 'expand', 'expect', 'expensive', 'experience', 'experiment', 'expert', 'explain', 'explode', 'express', 'eye', 'eyeglasses', 'eyes', 'f', 'face', 'fact', 'faculty', 'fail', 'fairy', 'fake', 'fall in love', 'familiar', 'family', 'famous', 'fancy', 'far', 'farm', 'farmer', 'fast', 'fat', 'father', 'fault', 'favorite', 'fear', 'february', 'federal', 'feed', 'feedback', 'feel', 'fence', 'festival', 'few', 'fight', 'final', 'finally', 'finance', 'find', 'fine', 'fingerspell', 'finish', 'fire', 'firefighter', 'first', 'fish', 'fishing', 'five', 'fix', 'flag', 'flatter', 'flexible', 'flirt', 'flood', 'floor', 'florida', 'flower', 'flute', 'fly', 'fold', 'follow', 'food', 'fool', 'football', 'for', 'forbid', 'forest', 'forever', 'forget', 'forgive', 'fork', 'form', 'former', 'four', 'fourth', 'fox', 'france', 'free', 'freeway', 'freeze', 'french', 'french fries', 'friday', 'friend', 'friendly', 'frog', 'from', 'from now on', 'front', 'fruit', 'full', 'fun', 'function', 'funeral', 'funny', 'furniture', 'future', 'g', 'gallaudet', 'gamble', 'game', 'gang', 'garage', 'gas', 'gasoline', 'gather', 'gay', 'general', 'generation', 'geography', 'geometry', 'german', 'germany', 'get', 'get up', 'ghost', 'gift', 'giraffe', 'girl', 'girlfriend', 'give', 'give up', 'glass', 'glasses', 'gloves', 'go', 'goal', 'goat', 'god', 'gold', 'golf', 'gone', 'good', 'goodbye', 'gorilla', 'gossip', 'government', 'grab', 'graduate', 'graduation', 'grammar', 'grandfather', 'grandma', 'grandmother', 'grapes', 'grass', 'grateful', 'gray', 'great', 'greece', 'green', 'grey', 'group', 'grow', 'grow up', 'guess', 'guide', 'guilty', 'guitar', 'gum', 'gun', 'gymnastics', 'h', 'habit', 'hair', 'haircut', 'half', 'halloween', 'hamburger', 'hammer', 'hang up', 'hanukkah', 'happen', 'happy', 'hard', 'hard of hearing', 'hat', 'hate', 'have', 'hawaii', 'head', 'headache', 'health', 'heap', 'hear', 'hearing', 'hearing aid', 'heart', 'heart attack', 'heaven', 'heavy', 'helicopter', 'hello', 'helmet', 'help', 'her', 'here', 'herself', 'hide', 'high', 'high school', 'highway', 'hill', 'hippopotamus', 'his', 'history', 'hit', 'hockey', 'hold', 'holy', 'home', 'homework', 'honest', 'honey', 'honor', 'hop', 'hope', 'horse', 'hospital', 'host', 'hot', 'hot dog', 'hour', 'house', 'how', 'hug', 'human', 'humble', 'hungry', 'hunt', 'hurricane', 'hurry', 'hurt', 'husband', 'i', 'ice cream', 'idea', 'identify', 'if', 'ignore', 'ill', 'illegal', 'image', 'impact', 'important', 'impossible', 'improve', 'in', 'include', 'increase', 'independent', 'india', 'individual', 'infection', 'influence', 'inform', 'information', 'innocent', 'insect', 'inside', 'inspect', 'inspire', 'instead', 'institute', 'insurance', 'interest', 'interesting', 'international', 'internet', 'interpret', 'interpreter', 'interrupt', 'intersection', 'interview', 'introduce', 'invest', 'investigate', 'invite', 'involve', 'iran', 'ireland', 'iron', 'island', 'israel', 'italy', 'j', 'jacket', 'jail', 'january', 'japan', 'jealous', 'jesus', 'jewelry', 'jewish', 'join', 'joke', 'journey', 'joy', 'judge', 'july', 'jump', 'june', 'k', 'kangaroo', 'karate', 'keep', 'key', 'keyboard', 'kick', 'kid', 'kill', 'kindergarten', 'king', 'kiss', 'kitchen', 'kneel', 'knife', 'knock', 'know', 'label', 'lady', 'lamp', 'land', 'language', 'laptop', 'large', 'last', 'last week', 'last year', 'late', 'later', 'laugh', 'laundry', 'law', 'lawyer', 'lazy', 'lead', 'leader', 'leaf', 'league', 'leak', 'learn', 'leave', 'lecture', 'left', 'legal', 'lemon', 'lend', 'lesbian', 'less', 'lesson', 'let', 'letter', 'lettuce', 'liability', 'librarian', 'library', 'license', 'lie', 'lift', 'light', 'lightning', 'like', 'limit', 'line', 'linguistics', 'lion', 'lip', 'lipstick', 'list', 'listen', 'little bit', 'live', 'loan', 'lobster', 'local', 'lock', 'lonely', 'long', 'look at', 'look for', 'lord', 'lose', 'loud', 'lousy', 'love', 'lucky', 'lunch', 'm', 'machine', 'mad', 'magazine', 'magic', 'mainstream', 'major', 'make', 'man', 'manage', 'manager', 'many', 'march', 'marry', 'match', 'math', 'mature', 'maximum', 'maybe', 'me', 'mean', 'measure', 'meat', 'mechanic', 'medicine', 'meet', 'meeting', 'melody', 'melt', 'member', 'memorize', 'mention', 'message', 'metal', 'mexico', 'microphone', 'microscope', 'microwave', 'middle', 'midnight', 'military', 'milk', 'mind', 'mine', 'minus', 'minute', 'mirror', 'miss', 'mistake', 'misunderstand', 'mix', 'mock', 'mom', 'monday', 'money', 'monkey', 'monster', 'month', 'monthly', 'moon', 'moose', 'more', 'morning', 'mosquito', 'most', 'mother', 'motivate', 'motor', 'motorcycle', 'mountain', 'mouse', 'mouth', 'move', 'movie', 'much', 'multiply', 'murder', 'muscle', 'museum', 'mushroom', 'music', 'must', 'mustache', 'my', 'myself', 'n', 'name', 'napkin', 'narrow', 'nation', 'near', 'necessary', 'neck', 'necklace', 'need', 'negative', 'negotiate', 'neighbor', 'nephew', 'nervous', 'network', 'neutral', 'never', 'new', 'new york', 'newspaper', 'next', 'nice', 'nickel', 'niece', 'night', 'nine', 'nineteen', 'no', 'none', 'noon', 'normal', 'north', 'northwest', 'nose', 'not', 'not yet', 'nothing', 'notice', 'november', 'now', 'number', 'numerous', 'nun', 'nurse', 'nut', 'o', 'objective', 'obsess', 'obtain', 'occur', 'ocean', 'october', 'octopus', 'odd', 'odor', 'off', 'offer', 'office', 'often', 'ok', 'old', 'olympics', 'on', 'once', 'one', 'onion', 'only', 'open', 'operate', 'opinion', 'opposite', 'or', 'oral', 'orange', 'order', 'organize', 'other', 'our', 'out', 'outside', 'over', 'overcome', 'overlook', 'overwhelm', 'owe', 'owl', 'p', 'pack', 'page', 'pain', 'paint', 'painter', 'pants', 'paper', 'parachute', 'parade', 'paragraph', 'parallel', 'parents', 'part', 'party', 'pass', 'past', 'path', 'patient', 'pause', 'pay', 'pay attention', 'peace', 'peaceful', 'peach', 'peanut butter', 'pear', 'peel', 'penalty', 'pencil', 'pennsylvania', 'penny', 'people', 'pepper', 'percent', 'perfect', 'perfume', 'period', 'permit', 'person', 'personality', 'perspective', 'pet', 'philadelphia', 'philosophy', 'phone', 'photographer', 'phrase', 'physician', 'physics', 'piano', 'pick', 'pickle', 'picture', 'pie', 'piece', 'pig', 'pillow', 'pilot', 'pink', 'pity', 'pizza', 'place', 'plan', 'plant', 'plate', 'play', 'player', 'please', 'plenty', 'plus', 'pneumonia', 'pocket', 'point', 'polar bear', 'police', 'policeman', 'policy', 'polite', 'politics', 'poop', 'poor', 'popcorn', 'popular', 'position', 'positive', 'possible', 'postpone', 'potato', 'potential', 'pound', 'pour', 'poverty', 'power', 'practice', 'praise', 'pray', 'preach', 'preacher', 'precious', 'precipitation', 'precise', 'predict', 'prefer', 'pregnant', 'prepare', 'preschool', 'present', 'presentation', 'president', 'pressure', 'pretty', 'prevent', 'price', 'pride', 'priest', 'prince', 'princess', 'principal', 'principle', 'print', 'printer', 'priority', 'prison', 'private', 'problem', 'process', 'procrastinate', 'professional', 'professor', 'profit', 'program', 'progress', 'project', 'promise', 'promote', 'proof', 'propaganda', 'proper', 'prostitute', 'protect', 'proud', 'prove', 'provide', 'psychologist', 'psychology', 'public', 'publish', 'pull', 'pumpkin', 'punish', 'purchase', 'pure', 'purple', 'purpose', 'pursue', 'push', 'put', 'put off', 'puzzled', 'q', 'quality', 'quarrel', 'quarter', 'queen', 'question', 'quick', 'quiet', 'quit', 'quote', 'r', 'rabbit', 'raccoon', 'race', 'radio', 'rage', 'rain', 'rainbow', 'rake', 'rat', 'rather', 'read', 'ready', 'real', 'realize', 'really', 'reason', 'receive', 'recent', 'recliner', 'recognize', 'recommend', 'recover', 'red', 'reduce', 'refer', 'referee', 'refuse', 'regular', 'rehearse', 'reject', 'relate', 'relationship', 'relax', 'release', 'relief', 'religion', 'rely', 'remember', 'remote control', 'remove', 'rent', 'repeat', 'replace', 'reply', 'report', 'represent', 'reputation', 'request', 'require', 'rescue', 'research', 'resign', 'resist', 'respect', 'responsibility', 'responsible', 'rest', 'restaurant', 'restroom', 'result', 'retire', 'retreat', 'reveal', 'revenge', 'review', 'rich', 'ride', 'ridiculous', 'right', 'ring', 'rise', 'river', 'road', 'roar', 'rob', 'robber', 'robot', 'rock', 'role', 'roof', 'room', 'roommate', 'rooster', 'rope', 'rose', 'rough', 'rubber', 'rude', 'ruin', 'rule', 'run', 'rush', 'russia', 's', 'sad', 'safe', 'salad', 'salary', 'salt', 'salute', 'same', 'sandwich', 'satisfy', 'saturday', 'sauce', 'sausage', 'save', 'saw', 'say', 'scan', 'scared', 'schedule', 'school', 'science', 'scientist', 'scissors', 'scold', 'score', 'scotland', 'scream', 'screwdriver', 'sculpture', 'sea', 'search', 'second', 'secret', 'secretary', 'see', 'seem', 'seldom', 'selfish', 'sell', 'senate', 'send', 'senior', 'sensitive', 'sentence', 'separate', 'september', 'sequence', 'serious', 'serve', 'service', 'settle', 'seven', 'several', 'sew', 'shame', 'shampoo', 'shape', 'share', 'shave', 'she', 'sheep', 'shelf', 'shine', 'shirt', 'shock', 'shoes', 'shoot', 'shop', 'shopping', 'short', 'should', 'shoulder', 'shout', 'shovel', 'show', 'shower', 'shrimp', 'shy', 'sick', 'side', 'sign', 'sign language', 'silent', 'silly', 'silver', 'similar', 'simple', 'sin', 'since', 'sing', 'singer', 'single', 'siren', 'sister', 'sit', 'situation', 'six', 'sixteen', 'size', 'skate', 'skeleton', 'sketch', 'ski', 'skill', 'skin', 'skinny', 'skip', 'skirt', 'skunk', 'sky', 'slave', 'sleep', 'sleepy', 'slice', 'slip', 'slow', 'small', 'smart', 'smell', 'smile', 'smoking', 'smooth', 'snack', 'snake', 'sneeze', 'snob', 'snow', 'snowman', 'soap', 'soccer', 'society', 'socks', 'soda', 'sofa', 'soft', 'soldier', 'solid', 'solve', 'some', 'someday', 'someone', 'something', 'sometimes', 'somewhere', 'son', 'song', 'soon', 'sore throat', 'sorry', 'soul', 'sound', 'soup', 'sour', 'south', 'south america', 'spain', 'spanish', 'speak', 'special', 'specific', 'speech', 'speed', 'spell', 'spider', 'spill', 'spin', 'spirit', 'spit', 'spoon', 'spray', 'spread', 'spring', 'sprint', 'square', 'squeeze', 'squirrel', 'stadium', 'staff', 'stairs', 'stamp', 'stand', 'standard', 'star', 'stare', 'start', 'statistics', 'stay', 'steal', 'steel', 'stepfather', 'sticky', 'still', 'sting', 'stink', 'stir', 'stitch', 'stomach', 'stop', 'store', 'story', 'straight', 'strange', 'straw', 'strawberry', 'street', 'stress', 'stretch', 'strict', 'strong', 'structure', 'struggle', 'stubborn', 'stuck', 'student', 'study', 'stupid', 'subtract', 'subway', 'sue', 'suffer', 'sugar', 'suggest', 'summer', 'summon', 'sun', 'sunday', 'sunrise', 'sunset', 'sunshine', 'superman', 'support', 'suppose', 'sure', 'surface', 'surgeon', 'surgery', 'surprise', 'surrender', 'suspect', 'suspend', 'swallow', 'sweater', 'sweden', 'sweep', 'sweet', 'sweetheart', 'swim', 'swimming', 'swimsuit', 'swing', 'switzerland', 'symbol', 'sympathy', 't', 'table', 'take', 'take turns', 'take up', 'tale', 'talent', 'talk', 'tall', 'tan', 'taste', 'tea', 'teach', 'teacher', 'team', 'tease', 'technology', 'teeth', 'telephone', 'telescope', 'television', 'tell', 'temperature', 'temple', 'tempt', 'ten', 'tend', 'tender', 'tennis', 'tent', 'terrible', 'test', 'testify', 'texas', 'text', 'thailand', 'than', 'thank you', 'thankful', 'thanksgiving', 'that', 'theater', 'their', 'them', 'theme', 'themselves', 'then', 'theory', 'therapy', 'there', 'therefore', 'thermometer', 'they', 'thick', 'thin', 'thing', 'think', 'third', 'thirsty', 'this', 'thousand', 'three', 'thrill', 'throat', 'through', 'throw', 'thursday', 'ticket', 'tie', 'tiger', 'time', 'tiptoe', 'tired', 'tissue', 'title', 'to', 'toast', 'tobacco', 'today', 'together', 'toilet', 'toilet paper', 'tolerate', 'tomato', 'tomorrow', 'tongue', 'tonight', 'tooth', 'toothbrush', 'top', 'topic', 'tornado', 'torture', 'total', 'touch', 'tough', 'tournament', 'towel', 'tower', 'town', 'trade', 'tradition', 'traffic', 'train', 'tranquil', 'transfer', 'transform', 'translate', 'travel', 'tree', 'triangle', 'trip', 'trophy', 'trouble', 'truck', 'true', 'trust', 'truth', 'try', 'tuesday', 'turkey', 'turn', 'turtle', 'tutor', 'tv', 'twin', 'two', 'type', 'u', 'ugly', 'umbrella', 'uncle', 'under', 'understand', 'underwear', 'unique', 'united states', 'university', 'until', 'up', 'upset', 'upstairs', 'use', 'v', 'vacant', 'vacation', 'vague', 'valley', 'value', 'vampire', 'vegetable', 'verb', 'very', 'vice president', 'viewpoint', 'violin', 'visit', 'visitor', 'visualize', 'vlog', 'vocabulary', 'voice', 'volleyball', 'volunteer', 'vomit', 'vote', 'w', 'wait', 'waiter', 'wake up', 'walk', 'wall', 'wallet', 'wander', 'want', 'war', 'warm', 'warn', 'wash', 'wash face', 'washington', 'waste', 'watch', 'water', 'waterfall', 'watermelon', 'way', 'we', 'weak', 'wear', 'weather', 'wedding', 'wednesday', 'week', 'weekend', 'weekly', 'weigh', 'weight', 'weird', 'welcome', 'west', 'wet', 'whale', 'what', 'whatever', 'wheelchair', 'when', 'where', 'which', 'while', 'whistle', 'white', 'who', 'why', 'wide', 'wife', 'will', 'willing', 'win', 'wind', 'window', 'wine', 'winter', 'wish', 'with', 'within', 'without', 'witness', 'wolf', 'woman', 'wonder', 'wonderful', 'wood', 'word', 'work', 'worker', 'workshop', 'world', 'worm', 'worry', 'worse', 'worthless', 'wow', 'wrap', 'wrench', 'wristwatch', 'write', 'wrong', 'year', 'yellow', 'yes', 'yesterday', 'you', 'young', 'your', 'yourself', 'zero'],  # Replace with actual class labels
    oov_token='unknown'  # Handling OOV values
)

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    # Convert class labels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

train_data, train_labels = prepare_all_videos(train_df, "Train")
test_data, test_labels = prepare_all_videos(test_df, "Test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"train_labels in train set: {train_labels.shape}")

Frame features in train set: (8313, 20, 2048)
Frame masks in train set: (8313, 20)
train_labels in train set: (8313, 1)


In [65]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
180/182 [============================>.] - ETA: 0s - loss: 7.5864 - accuracy: 0.0010    
Epoch 1: val_loss improved from inf to 7.70677, saving model to ./tmp\video_classifier
182/182 [==============================] - 17s 64ms/step - loss: 7.5863 - accuracy: 0.0010 - val_loss: 7.7068 - val_accuracy: 0.0000e+00
Epoch 2/30
181/182 [============================>.] - ETA: 0s - loss: 7.5318 - accuracy: 0.0021
Epoch 2: val_loss did not improve from 7.70677
182/182 [==============================] - 6s 34ms/step - loss: 7.5318 - accuracy: 0.0021 - val_loss: 7.8347 - val_accuracy: 0.0000e+00
Epoch 3/30
182/182 [==============================] - ETA: 0s - loss: 7.4895 - accuracy: 0.0012
Epoch 3: val_loss did not improve from 7.70677
182/182 [==============================] - 10s 56ms/step - loss: 7.4895 - accuracy: 0.0012 - val_loss: 7.9599 - val_accuracy: 0.0000e+00
Epoch 4/30
181/182 [============================>.] - ETA: 0s - loss: 7.4540 - accuracy: 0.0017
Epoch 4: val_loss did

In [66]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: dataset\frames\Test/necklace/66162
1/1 [==============================] - 3s 3s/step
  bed:  0.06%
  help:  0.06%
  cool:  0.06%
  go:  0.06%
  learn:  0.05%
  dry:  0.05%
  dark:  0.05%
  before:  0.05%
  corn:  0.05%
  cousin:  0.05%
  discuss:  0.05%
  cry:  0.05%
  graduate:  0.05%
  because:  0.05%
  meet:  0.05%
  candy:  0.05%
  headache:  0.05%
  letter:  0.05%
  appointment:  0.05%
  can:  0.05%
  different:  0.05%
  make:  0.05%
  full:  0.05%
  australia:  0.05%
  mother:  0.05%
  picture:  0.05%
  cop:  0.05%
  last:  0.05%
  dog:  0.05%
  argue:  0.05%
  carrot:  0.05%
  daughter:  0.05%
  hair:  0.05%
  family:  0.05%
  onion:  0.05%
  hope:  0.05%
  rabbit:  0.05%
  dissolve:  0.05%
  balloon:  0.05%
  barely:  0.05%
  new:  0.05%
  city:  0.05%
  match:  0.05%
  bell:  0.05%
  deodorant:  0.05%
  buy:  0.05%
  motorcycle:  0.05%
  pull:  0.05%
  mirror:  0.05%
  basketball:  0.05%
  no:  0.05%
  cook:  0.05%
  many:  0.05%
  drink:  0.05%
  egg:  0.05%
